<a href="https://colab.research.google.com/github/AlukweJonesTerah/A-simple-website/blob/main/coffee_startup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install kaggle
#!pip install tensorflow
#!pip install pillow

In [ ]:
import cv2
import PIL
import shutil
import random
import pathlib
import numpy as np
import pandas as pd
import os, warnings
from PIL import Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, RandomFlip, RandomRotation, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import drive, files
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix


In [ ]:
#mount drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

coffee_dataset_path = '/content/drive/MyDrive/coffee_dataset'
folder_contents = os.listdir (coffee_dataset_path)
print("contents of coffee dataset: ", folder_contents)

contents of coffee dataset:  ['healthy_coffee', 'Cerscospora_leaf_spot', 'Phoma_leaf_spot']


In [ ]:
data_dir = '/content/drive/MyDrive/coffee_dataset/'
#data_dir = pathlib.Path(data_dir).with_suffix('')

In [ ]:
bad_img_list=[]
total=0
good=0
bad=0

classes=sorted(os.listdir(data_dir))

print(classes)
for klass in classes:
    good_class=0
    bad_class=0
    total_class=0
    msg=f'processing class {klass}'
    print(msg, '\r', end= '')
    classpath=os.path.join(data_dir, klass)
    flist=sorted(os.listdir(classpath))
    for f in flist:
        total +=1
        total_class +=1
        fpath=os.path.join(classpath,f)
        try:
            img= Image.open(fpath)
            array=np.asarray(img)
            good +=1
            good_class +=1
        except:
            bad_img_list.append(fpath)
            bad +=1
            bad_class +=1

    msg=f'class {klass} contains {total_class} files, {good_class} are valid image files and {bad_class} defective image files'
    print (msg)
msg=f'the dataset contains {total} image files, {good} are valid image files and {bad} are defective image files'
print (msg)
if bad>0:
    ans=input('to print a list of defective image files enter P, to not print press Enter')
    if ans == 'P' or ans == 'p':
        for f in bad_img_list:
            print (f)



['Cerscospora_leaf_spot', 'Phoma_leaf_spot', 'healthy_coffee']
class Cerscospora_leaf_spot contains 7701 files, 7701 are valid image files and 0 defective image files
class Phoma_leaf_spot contains 6591 files, 6591 are valid image files and 0 defective image files


In [ ]:

train_data_dir = data_dir
test_data_dir = data_dir


train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)


test_datagen = ImageDataGenerator( rescale = 1.0/255. )




In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(data_dir, batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( data_dir,  batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

Found 18618 images belonging to 3 classes.
Found 18618 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final softmax layer with 1 node for classification output
x = layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath="best_model.h5",
    save_best_only=True,
    monitor="val_loss",
    mode="min",
    verbose=1
)
early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)
reduce_lr_callback=ReduceROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=2,
    min_lr=0.0001
)

In [ ]:
vgghist = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    epochs = 10,
    callback=[
        model_checkpoint_callback,
        early_stopping_callback,
        reduce_lr_callback
    ]
    )


Epoch 1/10
100/100 [==============================] - 197s 2s/step - loss: 0.5722 - accuracy: 0.7705 - val_loss: 0.1614 - val_accuracy: 0.9669
Epoch 2/10
100/100 [==============================] - 125s 1s/step - loss: 0.2467 - accuracy: 0.9025 - val_loss: 0.2966 - val_accuracy: 0.8606
Epoch 3/10
100/100 [==============================] - 126s 1s/step - loss: 0.1893 - accuracy: 0.9274 - val_loss: 0.0476 - val_accuracy: 0.9901
Epoch 4/10
100/100 [==============================] - 174s 2s/step - loss: 0.1347 - accuracy: 0.9540 - val_loss: 0.0739 - val_accuracy: 0.9875
Epoch 5/10
100/100 [==============================] - 174s 2s/step - loss: 0.1211 - accuracy: 0.9510 - val_loss: 0.0356 - val_accuracy: 0.9931
Epoch 6/10
100/100 [==============================] - 124s 1s/step - loss: 0.1282 - accuracy: 0.9555 - val_loss: 0.0344 - val_accuracy: 0.9875
Epoch 7/10
100/100 [==============================] - 127s 1s/step - loss: 0.1034 - accuracy: 0.9615 - val_loss: 0.0454 - val_accuracy: 0.9793

In [ ]:
model.summary()
model.save('model_1.h5')